**In this notebook i will implement Articial Neural Network from scratch**

In [1]:
import numpy as np

In [2]:
def initiliaze_parameters(input_size, hidden_size, output_size):
  np.random.seed(42)
  parameters = {
      "W1": np.random.randn(hidden_size, input_size) * 0.01,
      "b1": np.zeros((hidden_size, 1)),
      "W2": np.random.randn(output_size, hidden_size) * 0.01,
      "b2": np.zeros((output_size, 1))
  }

  return parameters

In [3]:
# these are activation function
def sigmoid(Z):
  return 1 / (1 + np.exp(-Z))

def relu(Z):
  return np.maximum(0, Z)

def relu_derivative(Z):
  return (Z > 0).astype(int)

In [4]:
def forward_propagation(X, parameters):
  W1 = parameters["W1"]
  b1 = parameters["b1"]
  W2 = parameters["W2"]
  b2 = parameters["b2"]

  Z1 = np.dot(W1, X) + b1
  A1 = relu(Z1)
  Z2 = np.dot(W2, A1) + b2
  A2 = sigmoid(Z2)

  cache = {
      "Z1": Z1,
      "A1": A1,
      "Z2": Z2,
      "A2": A2
  }

  return A2, cache

In [5]:
#Loss function

def compute_cost(Y, A2):
  m = Y.shape[1]
  cost = -1 / m * np.sum(Y * np.log(A2) + (1-Y) * np.log(1-A2))
  return np.squeeze(cost)

In [6]:
def backward_propagation(X, Y, parameters, cache):
    m = X.shape[1]
    W2 = parameters["W2"]

    dZ2 = cache["A2"] - Y
    dW2 = np.dot(dZ2, cache["A1"].T) / m
    db2 = np.sum(dZ2, axis=1, keepdims=True) / m

    dZ1 = np.dot(W2.T, dZ2) * relu_derivative(cache["Z1"])
    dW1 = np.dot(dZ1, X.T) / m
    db1 = np.sum(dZ1, axis=1, keepdims=True) / m

    grads = {"dW1": dW1, "db1": db1, "dW2": dW2, "db2": db2}
    return grads

In [7]:
def update_parameters(parameters, grads, learning_rate):
  for key in parameters.keys():
    parameters[key] = parameters[key] - learning_rate * grads["d"+key]
  return parameters

In [8]:

def train_network(X, y, input_size, hidden_size, output_size, epochs=1000, learning_rate=0.01):
  parameters = initiliaze_parameters(input_size, hidden_size, output_size)

  for i in range(epochs):
    A2, cache = forward_propagation(X, parameters)
    cost = compute_cost(y, A2)
    grads = backward_propagation(X, y, parameters, cache)
    parameters = update_parameters(parameters, grads, learning_rate)

    if i % 100 == 0:
      print(f"Epoch {i}: Cost = {cost}")

  return parameters

In [9]:
def predict(X, parameters):
  A2, _ = forward_propagation(X, parameters)
  return (A2 > 0.5).astype(int)

In [14]:
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
data = load_breast_cancer()
X = data.data        # features
y = data.target      # labels
print("X shape:", X.shape)
print("y shape:", y.shape)


X shape: (569, 30)
y shape: (569,)


In [15]:
scaler = StandardScaler()
X = scaler.fit_transform(X)


In [17]:
X_train_raw, X_test_raw, y_train_raw, y_test_raw = train_test_split(
    X,      # X is already (samples, features)
    y.reshape(-1, 1),  # convert to column vector (m,1)
    test_size=0.2,
    random_state=42
)

# Transpose to match your network's expected shape (features, samples)
X_train = X_train_raw.T  # (features, m_train)
X_test = X_test_raw.T    # (features, m_test)
y_train = y_train_raw.T  # (1, m_train)
y_test = y_test_raw.T    # (1, m_test)


In [18]:
input_size = X_train.shape[0]   # 30 features
hidden_size = 16                # you can adjust
output_size = 1                 # binary output

parameters = train_network(
    X_train, y_train,
    input_size=input_size,
    hidden_size=hidden_size,
    output_size=output_size,
    epochs=2000,
    learning_rate=0.01
)


Epoch 0: Cost = 0.6927889491881857
Epoch 100: Cost = 0.67750744628966
Epoch 200: Cost = 0.6610310768587488
Epoch 300: Cost = 0.6228173991122861
Epoch 400: Cost = 0.52243479862425
Epoch 500: Cost = 0.37931562777680067
Epoch 600: Cost = 0.27146946502065916
Epoch 700: Cost = 0.2080548662272049
Epoch 800: Cost = 0.17070696445890796
Epoch 900: Cost = 0.14612914165435276
Epoch 1000: Cost = 0.12870373269541124
Epoch 1100: Cost = 0.1161273832715347
Epoch 1200: Cost = 0.10654458512738975
Epoch 1300: Cost = 0.09893204804905578
Epoch 1400: Cost = 0.09295633801607649
Epoch 1500: Cost = 0.08811883072484435
Epoch 1600: Cost = 0.08409957050311466
Epoch 1700: Cost = 0.08076598504059339
Epoch 1800: Cost = 0.07795276451384504
Epoch 1900: Cost = 0.075522895769023


In [19]:
A2_train, _ = forward_propagation(X_train, parameters)
preds_train = (A2_train > 0.5).astype(int)
accuracy_train = np.mean(preds_train == y_train)
print("Training Accuracy:", accuracy_train)


Training Accuracy: 0.9824175824175824


In [20]:
A2_test, _ = forward_propagation(X_test, parameters)
preds_test = (A2_test > 0.5).astype(int)
accuracy_test = np.mean(preds_test == y_test)
print("Test Accuracy:", accuracy_test)


Test Accuracy: 0.9912280701754386
